In [2]:
link1, identifier1 = "https://legislatie.just.ro/Public/DetaliiDocument/164782", "Regulament"
link2, identifier2 = "https://legislatie.just.ro/Public/DetaliiDocument/216961", "OUG"
link3, identifier3 = "https://legislatie.just.ro/Public/DetaliiDocument/190029", "RCA"
link4, identifier4 = "https://legislatie.just.ro/Public/DetaliiDocument/24006", "ITP"
link5, identifier5 = "https://legislatie.just.ro/Public/DetaliiDocument/223635", "PENAL"

In [ ]:
import os
import requests

def download_html(link, output_folder, filename):
    html = requests.get(link).content
    filename = filename + ".html"
    output_path = os.path.join(output_folder, filename)
    with open(output_path, "wb") as f:
        f.write(html)
    return output_path

download_html(link1, "../repo/data/raw_data/law_corpus" ,identifier1)
download_html(link2, "../repo/data/raw_data/law_corpus" ,identifier2)
download_html(link3, "../repo/data/raw_data/law_corpus" ,identifier3)
download_html(link4, "../repo/data/raw_data/law_corpus" ,identifier4)
download_html(link5, "../repo/data/raw_data/law_corpus" ,identifier5)

'../repo/data/raw_data/law_corpus/PENAL.html'

In [6]:
import requests
from bs4 import BeautifulSoup
import re

def parse_corpus(link, identifier):
    results = []


    html = requests.get(link).content
    soup = BeautifulSoup(html, "html.parser")
    main_data = soup.find("div", id="div_Formaconsolidata")
    for cap in main_data.find_all("span", class_="S_CAP"):
        # print(cap)
        name = cap.find("span", class_="S_CAP_DEN").get_text(" ")
        body = cap.find("span", class_="S_CAP_BDY")
        for article in body.find_all("span", class_="S_ART"):
            number = article.find("span", class_="S_ART_TTL").get_text(" ").split(" ")[-1].replace('^','.')
            content_span = article.find("span", class_="S_ART_BDY")

            # Remove blue-colored elements
            for blue in content_span.find_all(style=lambda x: x and "color:blue" in x):
                if "abrogat" in blue.get_text(strip=True).lower():
                    blue.decompose()

            for a_tag in content_span.find_all("a"):
                a_tag.decompose()

            content = content_span.get_text(" ").replace('...', '')

            content = re.sub(r"\(la \d{2}-\d{2}-\d{4}.*?\)\s*\n", "", content, flags=re.DOTALL)

            title_metadata = f"{name}"

            sec_body = article.find_parent("span", class_="S_SEC_BDY")
            if sec_body:
                sec_title_span = sec_body.find_previous_sibling("span", class_="S_SEC_DEN")
                sec_title = sec_title_span.get_text(" ") if sec_title_span else None
                if sec_title != None:
                    title_metadata += f" | {sec_title}"
            
            # title_metadata += f" | Articolul {number}"
            title_metadata = title_metadata.strip()

            id = f"{identifier}-{number}"
            if content != None or content != None:
                results.append({"id": id, "title_metadata": title_metadata, "content": content})

    return results

def parse_corpus_itp(link, identifier):
    results = []


    html = requests.get(link).content
    soup = BeautifulSoup(html, "html.parser")
    main_data = soup.find("div", id="div_Formaconsolidata")

    for article in main_data.find_all("span", class_="S_ART"):
        number = article.find("span", class_="S_ART_TTL").get_text(" ").split(" ")[-1].replace('^','.')
        content_span = article.find("span", class_="S_ART_BDY")

        # Remove blue-colored elements
        for blue in content_span.find_all(style=lambda x: x and "color:blue" in x):
            if "abrogat" in blue.get_text(strip=True).lower():
                blue.decompose()

        for a_tag in content_span.find_all("a"):
            a_tag.decompose()

        content = content_span.get_text(" ").replace('...', '')

        content = re.sub(r"\(la \d{2}-\d{2}-\d{4}.*?\)\s*\n", "", content, flags=re.DOTALL)

        title_metadata = f"ITP"

        sec_body = article.find_parent("span", class_="S_SEC_BDY")
        if sec_body:
            sec_title_span = sec_body.find_previous_sibling("span", class_="S_SEC_DEN")
            sec_title = sec_title_span.get_text(" ") if sec_title_span else None
            if sec_title != None:
                title_metadata += f" | {sec_title}"
        
        # title_metadata += f" | Articolul {number}"
        title_metadata = title_metadata.strip()

        id = f"{identifier}-{number}"
        if content != None or content != None:
            results.append({"id": id, "title_metadata": title_metadata, "content": content})

    return results

In [7]:
# res1 = parse_corpus(link1, identifier1)
# res2 = parse_corpus(link2, identifier2)
# res3 = parse_corpus(link3, identifier3)
res4 = parse_corpus_itp(link4, identifier4)

In [8]:
# print(res1)
# print(res2)
# print(res3)
print(res4)

[{'id': 'ITP-1', 'title_metadata': 'ITP', 'content': '(1)    Prezenta ordonanță stabilește cadrul general de organizare și funcționare a sistemului de inspecție tehnică periodică a vehiculelor înmatriculate sau înregistrate în România. (2)    Cerințele specifice privind organizarea și funcționarea sistemului de inspecție tehnică periodică a vehiculelor înmatriculate sau înregistrate în România, precum și metodologia de efectuare a inspecției tehnice periodice se stabilesc prin ordin al ministrului transporturilor pentru aprobarea Reglementărilor privind inspecția tehnică periodică a vehiculelor înmatriculate sau înregistrate în România - RNTR 1 și se actualizează ori de câte ori este nevoie. (3)    În sensul prezentei ordonanțe, termenii și expresiile de mai jos au următoarele semnificații: a)   vehicul - orice autovehicul care nu circulă pe șine sau remorca sa; \xa0\xa0 b)   autovehicul - orice vehicul cu motor cu roți care se deplasează prin propriile mijloace; \xa0\xa0 c)   remorcă 

In [79]:
import pandas as pd
import ast

extracted_data = pd.read_csv('./test_sets/without_images_and_with_legislation.csv')
all_penal = []

for idx, item in extracted_data.iterrows():
    entry = ast.literal_eval(item['legislation'])
    print
    for obj in entry:
        if "PENAL" in obj:
            all_penal.append(obj)

all_penal = list(set(all_penal))
all_penal.sort()

In [82]:
res5 = parse_corpus(link5, identifier5)

In [83]:
filtered_res5 = [entry for entry in res5 if entry["id"] in all_penal]

In [9]:
import pandas as pd

df = pd.DataFrame(res4)
df.to_csv('./testcorpus.csv', index=False)